# Rough example usage

In [1]:
%load_ext autoreload
%autoreload 2

from blackboxbandits import compare, bandits, meta, utils
import pandas as pd

pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:20,.2f}'.format

## Base optimizer comparison

### Run a new comparison

In [ ]:
import pmlb
dataset_names = list(map(lambda x: "clf-"+x.replace("_","-"), pmlb.classification_dataset_names)) \
                + list(map(lambda x: "reg-"+x.replace("_","-"), pmlb.regression_dataset_names))

In [ ]:
base_comparison = compare.BaseOptimizerComparison(
    optimizers=["HyperOpt", "OpenTuner-BanditA", "OpenTuner-GA",
                "OpenTuner-GA-DE", "PySOT", "RandomSearch",
                "Scikit-GBRT-Hedge", "Scikit-GP-Hedge", "Scikit-GP-LCB"],
    classifiers=["MLP-adam","lasso"],
    datasets = dataset_names,
    metrics=["mse", "nll"],
    num_calls=20,
    num_repetitions=1,
    db_root = "./saved_results",
    datasets_root="./penn_datasets",
    parallel = True
)

In [ ]:
base_comparison.run()
dbid = base_comparison.get_dbid()
dbid

### View data from a previous comparison

In [2]:
DBID = "bo_20220228_124924_b7rkpeqg"

In [ ]:
df = compare.BaseOptimizerComparison.get_results_for_dbid(db_root="./saved_results", dbid=DBID)

## Meta optimizer comparison

### Set up meta optimizers

In [3]:
S = 1
GAMMA = 0.2

In [10]:
factors = [(a,b) for a in range(1,7) for b in range(1,7) if a*b < 7]

best = {
    f"best_{T}": meta.BestFixedTAlgos(T=T)
    for T in range(2,7)}
fpml_fixedexp = {
    f"fpml_fixedexp_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLFixed,
        T=T, S=S)
    for T in range(2,7)}
fpml_probexp = {
    f"fpml_probexp_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLProb,
        T=T, gamma=GAMMA)
    for T in range(2,7)}
fpml_gr_noexp = {
    f"fpml_gr_noexp_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLWithGR,
        T=T, gamma=0)
    for T in range(2,7)}
fpml_gr_probexp = {
    f"fpml_gr_probexp_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLWithGR,
        T=T, gamma=GAMMA)
    for T in range(2,7)}
streeter_fpml = {
    f"streeter_fpml_{a}x{b}": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML,
        T=a*b, T_1=a, T_2=b, gamma=GAMMA, gr=True)
    for a,b in factors}
streeter_exp3 = {
    f"streeter_exp3_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.Streeter,
        T=T)
    for T in range(2,7)}

meta_optimizers = {**fpml_fixedexp, **fpml_probexp, **fpml_gr_noexp,
                   **fpml_gr_probexp, **streeter_fpml, **streeter_exp3, **best}

### Setup and run meta comparison

In [13]:
meta_comparison = compare.MetaOptimizerComparison \
                         .from_precomputed_base_comparison(
    dbid=DBID,
    meta_optimizers=meta_optimizers,
    db_root = "./saved_results",
    parallel_meta = False,
    num_meta_repetitions = 1
)

In [14]:
meta_comparison.run_meta_comparison()
results = meta_comparison.summary()
full_details = meta_comparison.full_results()
results.to_csv("results.csv")
full_details.to_csv("full_details.csv")
results

Starting meta-comparison, repetition 1 of 1
Running meta-optimizer fpml_fixedexp_2 on base study 1 of 1
Running meta-optimizer fpml_fixedexp_3 on base study 1 of 1
Running meta-optimizer fpml_fixedexp_4 on base study 1 of 1
Running meta-optimizer fpml_fixedexp_5 on base study 1 of 1
Running meta-optimizer fpml_fixedexp_6 on base study 1 of 1
Running meta-optimizer fpml_probexp_2 on base study 1 of 1
Running meta-optimizer fpml_probexp_3 on base study 1 of 1
Running meta-optimizer fpml_probexp_4 on base study 1 of 1
Running meta-optimizer fpml_probexp_5 on base study 1 of 1
Running meta-optimizer fpml_probexp_6 on base study 1 of 1
Running meta-optimizer fpml_gr_noexp_2 on base study 1 of 1
Running meta-optimizer fpml_gr_noexp_3 on base study 1 of 1
Running meta-optimizer fpml_gr_noexp_4 on base study 1 of 1
Running meta-optimizer fpml_gr_noexp_5 on base study 1 of 1
Running meta-optimizer fpml_gr_noexp_6 on base study 1 of 1
Running meta-optimizer fpml_gr_probexp_2 on base study 1 of 1

KeyError: 'optimizer'

In [18]:
import pickle
pickle.dump(meta_comparison, open("meta_comparison.p", "wb"))

In [ ]:
utils.visualise_by_budget(results)